Optimizing hospital procedures - predicting medical costs based on patient data
===
---

In [ ]:
# ? imports and settings

---
# Executive summary

---
# Input data and transformations

---
# Exploratory data analysis

---
# Model building
# ? feature engineering / training / validation

---
# Findings

---
# Conclusions